### RESULTS:

##### RGB 
- means = array([74.90059245, 85.25801497, 80.06076522])
- stdevs = array([15.05452957, 13.87736375, 12.01005956])

##### DEM 
- means = array([608.951834])
- stdevs = array([2.30201424])


##### HS 
- means = array([179.17519397])
- stdevs = array([10.65291866])

### CODE FOR CALCULATING DATASET-WIDE MEAN AND STD 

In [7]:
#######
# RGB #
#######

import os
import rasterio 
import numpy as np

dir = "/home/nadjaflechner/Palsa_data/generated_datasets/FINAL_200m_dataset/rgb/"
all_rgb = os.listdir(dir)
all_rgb = [file for file in all_rgb if not file.endswith('aug.tif')]

img_means_sum = np.array([0.0,0.0,0.0])
img_std_sum = np.array([0.0,0.0,0.0])


for img in all_rgb:
    with rasterio.open(os.path.join(dir, img)) as img:
        img = img.read()
    img_means_sum += img.mean(axis=(1,2))
    img_std_sum += img.std(axis=(1,2))


means = img_means_sum / len(all_rgb)
stds = img_std_sum / len(all_rgb)

In [8]:
means

array([74.90059245, 85.25801497, 80.06076522])

In [9]:
stds

array([15.05452957, 13.87736375, 12.01005956])

In [10]:
##########
# dem #
##########

import os
import rasterio 
import numpy as np

dir = "/home/nadjaflechner/Palsa_data/generated_datasets/FINAL_200m_dataset/dem/"
all_rgb = os.listdir(dir)
all_rgb = [file for file in all_rgb if not file.endswith('aug.tif')]

img_means_sum = np.array([0.0])
img_std_sum = np.array([0.0])


for img in all_rgb:
    with rasterio.open(os.path.join(dir, img)) as img:
        img = img.read()
    img_means_sum += img.mean()
    img_std_sum += img.std()

dem_means = img_means_sum / len(all_rgb)
dem_stds = img_std_sum / len(all_rgb)

In [12]:
dem_stds

array([2.30201424])

In [13]:
##########
# hs #
##########

import os
import rasterio 
import numpy as np

dir = "/home/nadjaflechner/Palsa_data/generated_datasets/FINAL_200m_dataset/hs/"
all_rgb = os.listdir(dir)
all_rgb = [file for file in all_rgb if not file.endswith('aug.tif')]

img_means_sum = np.array([0.0])
img_std_sum = np.array([0.0])


for img in all_rgb:
    with rasterio.open(os.path.join(dir, img)) as img:
        img = img.read()
    img_means_sum += img.mean()
    img_std_sum += img.std()

hs_means = img_means_sum / len(all_rgb)
hs_stds = img_std_sum / len(all_rgb)

In [14]:
hs_means

array([179.17519397])

In [15]:
hs_stds

array([10.65291866])

### Messing around: 

In [69]:
############
# Imports #
############

import torch
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import rasterio
import numpy as np
from utils import ImageDataset, SaveFeatures, filter_dataset, accuracy, imshow_transform
from custom_model import model_4D
from torch.autograd import Variable
# from skimage.transform import resize
# from skimage.io import imshow
# import wandb
import matplotlib.pyplot as plt 
import torch.optim.lr_scheduler as lr_scheduler
import torchmetrics
import json

##################
## load configs ##
##################

config_path = os.path.join(os.getcwd(), 'configs.json')
with open(config_path, 'r') as config_file:
    configs = json.load(config_file)

# load paths configs dictionary
config_paths = configs.get('paths', {}) 

# assign paths
palsa_shapefile = config_paths.get('palsa_shapefile') 
parent_dir = config_paths.get('data') 
rgb_dir = os.path.join(parent_dir, 'rgb')
hs_dir = os.path.join(parent_dir, 'hs')
dem_dir = os.path.join(parent_dir, 'dem')
labels_file = os.path.join(parent_dir, 'palsa_labels.csv')

# load hyperparams configs dictionary
config_hyperparams = configs.get('hyperparams', {}) 

# assign hyperparams
n_samples = config_hyperparams.get('n_samples')
batch_size = config_hyperparams.get('batch_size')
num_epochs = config_hyperparams.get('num_epochs')
lr = config_hyperparams.get('lr')
lr_gamma = config_hyperparams.get('lr_gamma')
weight_decay = config_hyperparams.get('weight_decay')

# load data configs dictionary
config_data = configs.get('data', {}) 

# assign data configs
im_size = config_data.get('im_size')
min_palsa_positive_samples = config_data.get('min_palsa_positive_samples')
low_pals_in_val = config_data.get('low_pals_in_val')
augment = config_data.get('augment')
normalize = config_data.get('normalize')
depth_layer = config_data.get('depth_layer')

##########################
# log hyperparams to w&b #
##########################

# run = wandb.init(
#     # Set the project where this run will be logged
#     project="VGG_CAMs",
#     # Track hyperparameters and run metadata
#     config={
#         "learning_rate": lr,
#         "lr_gamma": lr_gamma,
#         "epochs": num_epochs,
#         "batch_size": batch_size,
#         "n_samples": n_samples,
#         "weight_decay": weight_decay,
#         "im_size": im_size,
#         "min_palsa_positive_samples": min_palsa_positive_samples,
#         "augment": augment,
#         "normalize": normalize,
#         "low_pals_in_val": low_pals_in_val,
#         "depth_layer": depth_layer
#             }#,
#     #tags=[]
# )

#########################
# configure dataloaders #
#########################

train_files, val_files = filter_dataset(labels_file, augment, min_palsa_positive_samples, low_pals_in_val, n_samples)

# choose depth data based on configs
depth_dir = hs_dir if depth_layer == "hs" else dem_dir

# Create the datasets and data loaders
train_dataset = ImageDataset(depth_dir, rgb_dir, train_files, im_size, normalize)
val_dataset = ImageDataset(depth_dir, rgb_dir, val_files, im_size, normalize)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

################
# define model #
################

model = model_4D()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=lr_gamma)
loss_function = nn.CrossEntropyLoss()

#######################
# model training loop #
#######################

# define metrics
Accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=2)
F1 = torchmetrics.F1Score(task="multiclass", num_classes=2)
Recall = torchmetrics.Recall(task="multiclass", average="micro", num_classes=2)

max_val_F1 = 0

for epoch in range(num_epochs):
    print('EPOCH: ',epoch+1)

    ############
    # training #
    ############

    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):     

        # load images and labels 
        images = Variable(images).to(device)  
        labels = Variable(labels.long()).to(device)  

        # train batch   
        outputs = model(images) 
        optimizer.zero_grad()
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()  

        # update metrics
        train_loss = loss.item()
        train_Accuracy = Accuracy(outputs.softmax(dim=-1), labels)
        train_Recall = Recall(outputs.softmax(dim=-1), labels)
        train_F1 = F1(outputs.softmax(dim=-1), labels)

    ##############
    # validation #
    ##############

    running_val_F1 = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(val_loader):  

            # load images and labels 
            images = Variable(images).to(device)  
            labels = Variable(labels.long()).to(device)  
            outputs = model(images) 
            loss = loss_function(outputs, labels)

            # update metrics
            val_loss = loss.item()
            val_Accuracy = Accuracy(outputs.softmax(dim=-1), labels)
            val_Recall = Recall(outputs.softmax(dim=-1), labels)

            # handle F1 separately for best model selection
            f1 = F1(outputs.softmax(dim=-1), labels)
            running_val_F1.append(f1.detach().cpu().numpy())
            val_F1 = f1

    # lr scheduler step 
    scheduler.step()

    # update current best model
    if np.mean(running_val_F1) > max_val_F1:
        best_model = model.state_dict()
        max_val_F1 = np.mean(running_val_F1)

# # after all epochs, save the best model as an artifact to wandb
# torch.save(best_model, '/home/nadjaflechner/models/model.pth')
# artifact = wandb.Artifact('model', type='model')
# artifact.add_file('/home/nadjaflechner/models/model.pth')
# run.log_artifact(artifact)

#################
# generate CAMs #
#################

EPOCH:  1


In [66]:
type(val_F1)

torch.Tensor

In [67]:
f1

tensor(0.)

In [68]:
f1.detach().cpu().numpy()

array(0., dtype=float32)

In [64]:
val_F1

tensor(0.)

In [27]:
import torch
import torch.nn as nn

avg_pool = nn.AdaptiveAvgPool2d(1)

tensie1 = torch.Tensor([[
    [1,2,3,4],
    [1,2,3,4],
    [1,2,3,4],
    [1,2,3,4]
]])
tensie2 = torch.cat((tensie1, tensie1))

In [29]:
avg_pool(tensie2)

tensor([[[2.5000]],

        [[2.5000]]])

In [22]:
tensie1.shape

torch.Size([4, 4])